<a href="https://colab.research.google.com/github/nibin101/intelligent_quiz/blob/main/intelligent_quiz_app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# STEP 0: Install dependencies

In [2]:
!pip install -qU google-generativeai==0.8.5 google-ai-generativelanguage==0.6.15\
langgraph langchain langchain-google-genai openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.5/152.5 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 785.6/785.6 kB 37.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 18.0 MB/s eta 0:00:00


# STEP 1: Imports and secure API key input

In [3]:
import os
import getpass


# STEP 2: Initialize Gemini 1.5 Flash

In [4]:
os.environ['GOOGLE_API_KEY']=getpass.getpass("Enter the Gemini api key")

Enter the Gemini api key··········



# Secure Gemini API Key input

In [5]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm=ChatGoogleGenerativeAI(model="models/gemini-1.5-flash-latest" ,temperature=0.2)

In [6]:
def get_subject_input(state: dict) -> dict:
    """Gets the subject input from the user."""
    subject = input("What subject do you want to learn about (physics, chemistry, or maths)? ")
    state["subject"] = subject.lower()
    return state

In [7]:
import json

def generate_mcqs(state: dict) -> dict:
    """Generates 5 MCQ questions on the given subject and stores them in the state."""
    subject = state["subject"]
    prompt = (
        f"Generate 5 multiple-choice questions about {subject}. "
        "Each question should have 4 options (A, B, C, D) and indicate the correct answer. "
        "Respond in JSON format as a list of objects, where each object has keys 'question', 'options', and 'answer'."
        "Do not include any ''' formatting"
    )
    response = llm.invoke([HumanMessage(content=prompt)])
    print(response.content)
    try:
        # Attempt to parse the content directly first
        questions_json = json.loads(response.content)
        state["questions"] = [(q["question"], q["options"], q["answer"]) for q in questions_json]
    except json.JSONDecodeError as e:
        print("Error decoding JSON response from the language model.")
        print(e)
        # If direct parsing fails, try stripping potential markdown formatting
        try:
            cleaned_response = response.content.strip().replace("```json\n", "").replace("```", "")
            questions_json = json.loads(cleaned_response)
            state["questions"] = [(q["question"], q["options"], q["answer"]) for q in questions_json]
        except json.JSONDecodeError as e_cleaned:
            print("Error decoding JSON response after cleaning.")
            print(e_cleaned)
            state["questions"] = []
    return state

In [ ]:
import json
import re
from langchain_core.messages import HumanMessage
from langchain_google_genai import ChatGoogleGenerativeAI

def get_subject_input(state: dict) -> dict:
    """Gets the subject input from the user."""
    subject = input("What subject do you want to learn about and be quizzed on? ")
    state["subject"] = subject.lower()
    return state

def generate_mcqs(state: dict) -> dict:
    """Generates 5 MCQ questions on the given subject and stores them in the state."""
    subject = state["subject"]
    prompt = (
        f"Generate 5 multiple-choice questions about {subject}. "
        "Each question should have 4 options (A, B, C, D) and indicate the correct answer. "
        "Respond in JSON format as a list of objects, where each object has keys 'question', 'options', and 'answer'."
        "Do not include any ''' formatting"
    )
    # Ensure llm is defined or imported if needed here
    # response = llm.invoke([HumanMessage(content=prompt)]) # Uncomment and ensure llm is available

    # Mock response for demonstration if llm is not available
    # response_content = """
    # [
    #     {
    #         "question": "What is the chemical symbol for water?",
    #         "options": {"A": "O2", "B": "H2O", "C": "CO2", "D": "NaCl"},
    #         "answer": "B"
    #     },
    #     {
    #         "question": "What is the capital of France?",
    #         "options": {"A": "Berlin", "B": "Madrid", "C": "Paris", "D": "Rome"},
    #         "answer": "C"
    #     },
    #     {
    #         "question": "What is the largest planet in our solar system?",
    #         "options": {"A": "Earth", "B": "Mars", "C": "Jupiter", "D": "Venus"},
    #         "answer": "C"
    #     },
    #     {
    #         "question": "What is the square root of 64?",
    #         "options": {"A": "6", "B": "7", "C": "8", "D": "9"},
    #         "answer": "C"
    #     },
    #     {
    #         "question": "Who wrote 'Romeo and Juliet'?",
    #         "options": {"A": "Charles Dickens", "B": "William Shakespeare", "C": "Jane Austen", "D": "Leo Tolstoy"},
    #         "answer": "B"
    #     }
    # ]
    # """
    # Replace the mock response with the actual LLM call when available
    try:
        llm=ChatGoogleGenerativeAI(model="models/gemini-1.5-flash-latest" ,temperature=0.2)
        response = llm.invoke([HumanMessage(content=prompt)])
        questions_json = json.loads(response.content)
        state["questions"] = [(q["question"], q["options"], q["answer"]) for q in questions_json]
    except json.JSONDecodeError as e:
        print("Error decoding JSON response from the language model.")
        print(e)
        try:
            cleaned_response = response.content.strip().replace("```json\n", "").replace("```", "")
            questions_json = json.loads(cleaned_response)
            state["questions"] = [(q["question"], q["options"], q["answer"]) for q in questions_json]
        except json.JSONDecodeError as e_cleaned:
            print("Error decoding JSON response after cleaning.")
            print(e_cleaned)
            state["questions"] = []

    # Using mock response for now
    # try:
    #     questions_json = json.loads(response_content)
    #     state["questions"] = [(q["question"], q["options"], q["answer"]) for q in questions_json]
    # except json.JSONDecodeError as e:
    #     print("Error decoding mock JSON response.")
    #     print(e)
    #     state["questions"] = []

    return state


def main():
    state = {}
    state = get_subject_input(state)

    while True:
        state = generate_mcqs(state)

        correct_answers = 0
        total_questions = len(state.get("questions", []))
        answered_correctly = []
        answered_incorrectly = []

        # Display MCQs and get user answers
        if state.get("questions"):
            print("\nHere are the questions:")
            for i, (question, options, answer) in enumerate(state["questions"]):
                # Remove <sup> tags from the question
                cleaned_question = re.sub(r'<sup[^>]*>.*?</sup>', '', question)
                print(f"\nQuestion {i+1}: {cleaned_question}")
                # Iterate through the options dictionary to display letter and text
                for letter, option_text in options.items():
                    # Remove <sup> tags from the options
                    cleaned_option_text = re.sub(r'<sup[^>]*>.*?</sup>', '', option_text)
                    print(f"{letter}. {cleaned_option_text}")

                user_answer = input("Your answer (enter the option letter, e.g., A): ").upper()

                # Check if the answer is correct
                # Simplified check based on the dictionary answer
                correct_option_letter = answer


                if user_answer == correct_option_letter:
                    print("Correct!")
                    correct_answers += 1
                    answered_correctly.append(question)
                else:
                    print(f"Incorrect. The correct answer is {correct_option_letter}.")
                    answered_incorrectly.append(question)
        else:
            print("No questions were generated.")

        # Print the score
        if total_questions > 0:
            print(f"\nYour score: {correct_answers}/{total_questions}")

            # Use LLM to summarize topics
            if answered_incorrectly:
                print("\nHere's a summary of the topics you answered incorrectly:")
                for incorrect_question in answered_incorrectly:
                    incorrect_topic_prompt = f"Explain the topic of the following question in a short paragraph:\n{incorrect_question}"
                    llm = ChatGoogleGenerativeAI(model="models/gemini-1.5-flash-latest" ,temperature=0.2)
                    incorrect_topic_response = llm.invoke([HumanMessage(content=incorrect_topic_prompt)])
                    cleaned_incorrect_topic_explanation = incorrect_topic_response.content.replace('**', '')
                    print(f"\n{cleaned_incorrect_topic_explanation}")

            # Ask for the next topic
            next_subject = input("\nWhich topic would you like to be quizzed on next? (or type 'quit' to exit) ")
            if next_subject.lower() == 'quit':
                break
            state["subject"] = next_subject.lower()
            print(f"Okay, I will generate questions about {next_subject} next.")


if __name__ == "__main__":
    main()

What subject do you want to learn about and be quizzed on? maths
Error decoding JSON response from the language model.
Expecting value: line 1 column 1 (char 0)

Here are the questions:

Question 1: What is the value of 3 + 4?
A. 12
B. 25
C. 36
D. 49
Your answer (enter the option letter, e.g., A): 69
Incorrect. The correct answer is B.

Question 2: Solve for x: 2x + 5 = 11
A. 3
B. 6
C. 8
D. 16
Your answer (enter the option letter, e.g., A): A
Correct!

Question 3: What is the area of a rectangle with length 8cm and width 5cm?
A. 13cm
B. 26cm
C. 40cm
D. 80cm
Your answer (enter the option letter, e.g., A): C
Correct!

Question 4: What is the next number in the sequence: 2, 4, 6, 8, ...?
A. 9
B. 10
C. 12
D. 16
Your answer (enter the option letter, e.g., A): D
Incorrect. The correct answer is B.

Question 5: What is the value of 15% of 200?
A. 10
B. 20
C. 30
D. 150
